# BMW processed time series Exploratory Data Analysis
The goal of this notebook is to elaborate a method to calculate the SoH of the BMW fleet.  

## Setup

### Imports

In [ ]:
from datetime import datetime as DT
import pytz

import pandas as pd
from pandas import DataFrame as DF
import plotly.express as px

from core.s3_utils import S3_Bucket
from core.config import *
from core.pandas_utils import *
from transform.processed_tss.main import get_processed_tss


### Data extraction

In [ ]:
tss = get_processed_tss("bmw")
tss.columns

In [ ]:
tss.head(5)

## Time series EDA

Let's list the variables and the respective count ratio.

In [ ]:
tss.count() / len(tss)


In [ ]:
VIN = "WBY1Z610407A12415"
tss_unique = tss.query("vin == @VIN")

random_vins = np.random.choice(tss['vin'].unique(), size=5, replace=False)

tss_sample = tss[tss['vin'].isin(random_vins)]

## Print first graphs

In [ ]:
px.scatter(tss, 
           x="soc", 
           y="estimated_range", 
           color="vin")

## SoH calculation

### First method on the estimated range


#### Few graphs


In [ ]:
px.scatter(tss, 
           x="soc", 
           y="estimated_range", 
           color="vin")

In [ ]:
px.scatter(tss_unique, 
           x="soc", 
           y="estimated_range", 
           color="in_charge")
